In [404]:
import sys
import os
import math

# Get the current working directory and the relative path to your module
module_path = os.path.abspath(os.path.join("..", ".."))
if module_path not in sys.path:
    sys.path.append(module_path)

# Now you can import your module
from src.api.service.repo import SearchRepo

In [405]:
es = SearchRepo()

Connected to Elasticsearch!
{'cluster_name': 'docker-cluster',
 'cluster_uuid': '9I9q3mFYSISbvBg75cK23A',
 'name': '5eeda383872f',
 'tagline': 'You Know, for Search',
 'version': {'build_date': '2024-08-05T10:05:34.233336849Z',
             'build_flavor': 'default',
             'build_hash': '1a77947f34deddb41af25e6f0ddb8e830159c179',
             'build_snapshot': False,
             'build_type': 'docker',
             'lucene_version': '9.11.1',
             'minimum_index_compatibility_version': '7.0.0',
             'minimum_wire_compatibility_version': '7.17.0',
             'number': '8.15.0'}}


In [406]:
idx = "vehicle"

In [407]:
query_string = "control"

In [ ]:
query = {
    "bool": {
        "filter": [
            {
                "nested": {
                    "_name": f"match_{q}",
                    "path": "completion_terms",
                    "query": {
                        "match": {
                            "completion_terms.tag.edge_ngram": {
                                "query": q,
                            }
                        }
                    },
                }
            }
            for q in query_string.split()
        ]
    }
}

In [409]:
query

{'bool': {'filter': [{'nested': {'_name': 'match_control',
     'path': 'completion_terms',
     'query': {'match': {'completion_terms.tag.edge_ngram': {'query': 'control'}}}}}]}}

In [410]:
resp = es.es.search(
    index=idx,
    query=query,
    size=1,
    source ="completion_terms",
    explain=True,
)
resp.body

{'took': 7,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 4031, 'relation': 'eq'},
  'max_score': 0.0,
  'hits': [{'_shard': '[vehicle][0]',
    '_node': 'PzEtOLwfQPOJs2xXldNFeQ',
    '_index': 'vehicle',
    '_id': '9cfbrpMBFDxTL3h7B-oZ',
    '_score': 0.0,
    '_source': {'completion_terms': [{'tag': 'Vehicle'},
      {'tag': 'Coupe'},
      {'tag': 'Lotus'},
      {'tag': 'Exige S'},
      {'tag': '2008'},
      {'tag': 'I4'},
      {'tag': '1.8L'},
      {'tag': 'Manual'},
      {'tag': 'RWD'},
      {'tag': 'Gasoline'},
      {'tag': 'Red'},
      {'tag': 'Standard'},
      {'tag': 'Standard'},
      {'tag': 'Leather seats'},
      {'tag': 'Navigation'},
      {'tag': 'Bluetooth'},
      {'tag': 'ABS'},
      {'tag': 'Airbags'},
      {'tag': 'Stability control'}]},
    'matched_queries': ['match_control'],
    '_explanation': {'value': 0.0,
     'description': 'ConstantScore(ToParentBlockJoinQuery (completi

In [106]:

def number_of_search_terms_from(query_string):
    """
    Returns the number of terms in the query string using space as a separator
    :param query_string: String containing zero or more words separated by spaces
    :return: Number with the amount of found terms
    """
    return len(query_string.split())


In [123]:
def get_completion_prefix(input_string):
    # if the input string is too short, then don't attempt completion
    if len(input_string) < 2:
        return None

    # get the last uncompleted string
    input_string = input_string.lstrip()
    last_space_index = input_string.rfind(" ")
    prefix = input_string[last_space_index + 1 :]

    # if the prefix is 1 or less chars then include the previous word in the prefix
    if len(prefix) <= 1:
        previous_space_index = input_string[:last_space_index].rfind(" ")
        prefix = input_string[previous_space_index + 1 :]

    return prefix


In [275]:
query_string = "Stability c"

In [257]:
number_of_search_terms_from(query_string)

2

In [321]:
query = None
if number_of_search_terms_from(query_string) <= 1:
    query = None
else:
    query = {
        "bool": {
            "filter": [
                {
                    "nested": {
                        "_name": f"match_{q}",
                        "path": "completion_terms",
                        "query": {
                            "match": {
                                "completion_terms.tag.edge_ngram": {
                                    "query": q,
                                }
                            }
                        },
                    }
                }
                for q in query_string.split()
            ]
        }
    }
query

{'bool': {'filter': [{'nested': {'_name': 'match_Stability',
     'path': 'completion_terms',
     'query': {'match': {'completion_terms.tag.edge_ngram': {'query': 'Stability'}}}}},
   {'nested': {'_name': 'match_c',
     'path': 'completion_terms',
     'query': {'match': {'completion_terms.tag.edge_ngram': {'query': 'c'}}}}}]}}

In [322]:
get_completion_prefix(query_string)

'Stability c'

In [323]:
number_of_search_terms_from(get_completion_prefix(query_string))

2

In [324]:
get_completion_prefix(query_string)

'Stability c'

In [325]:
aggs = {
    "topics_parent": {
        "nested": {"path": "completion_terms"},
        "aggs": {
            "topics": {
                "filter": {
                    "bool": {
                        "should": [
                            {
                                "match_phrase_prefix": {
                                    "completion_terms.tag": get_completion_prefix(query_string)
                                }
                            },
                        ]
                    }
                },
                "aggs": {
                    "topics": {
                        "terms": {
                            "field": "completion_terms.tag.keyword",
                            "size": 10,
                        },
                    }
                },
            }
        },
    }
}

aggs

{'topics_parent': {'nested': {'path': 'completion_terms'},
  'aggs': {'topics': {'filter': {'bool': {'should': [{'match_phrase_prefix': {'completion_terms.tag': 'Stability c'}}]}},
    'aggs': {'topics': {'terms': {'field': 'completion_terms.tag.keyword',
       'size': 10}}}}}}}

In [326]:
resp = es.es.search(
    index=idx,
    query=query,
    aggs=aggs,
    size=1,
    #filter_path="took,**.buckets.key,**.buckets.doc_count",
)
resp.body

{'took': 31,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 0, 'relation': 'eq'},
  'max_score': None,
  'hits': []},
 'aggregations': {'topics_parent': {'doc_count': 0,
   'topics': {'doc_count': 0,
    'topics': {'doc_count_error_upper_bound': 0,
     'sum_other_doc_count': 0,
     'buckets': []}}}}}

In [276]:
get_completion_prefix(query_string)

'Stability c'

In [319]:
query = {
    "nested": {
        "path": "completion_terms",
        "query": {
            "multi_match": {
                "query": get_completion_prefix(query_string),
                "operator": "and",
                "type": "phrase_prefix",
                "fields": [
                    "completion_terms.tag.as_you_type",
                    "completion_terms.tag.as_you_type._2gram",
                    "completion_terms.tag.as_you_type._3gram",
                ],
            }
        },
    }
}

In [320]:
resp = es.es.search(
    index=idx,
    query=query,
    size=1,
    source="completion_terms",
    #filter_path="took,**.buckets.key,**.buckets.doc_count",
    explain=True,
)
resp.body

{'took': 17,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 3704, 'relation': 'eq'},
  'max_score': 5.2239494,
  'hits': [{'_shard': '[vehicle][0]',
    '_node': 'PzEtOLwfQPOJs2xXldNFeQ',
    '_index': 'vehicle',
    '_id': 'bceGrpMBFDxTL3h7wtcp',
    '_score': 5.2239494,
    '_source': {'completion_terms': [{'tag': 'Vehicle'},
      {'tag': 'Coupe'},
      {'tag': 'Lotus'},
      {'tag': 'Exige S'},
      {'tag': '2008'},
      {'tag': 'I4'},
      {'tag': '1.8L'},
      {'tag': 'Manual'},
      {'tag': 'RWD'},
      {'tag': 'Gasoline'},
      {'tag': 'Red'},
      {'tag': 'Standard'},
      {'tag': 'Standard'},
      {'tag': 'Leather seats'},
      {'tag': 'Navigation'},
      {'tag': 'Bluetooth'},
      {'tag': 'ABS'},
      {'tag': 'Airbags'},
      {'tag': 'Stability control'}]},
    '_explanation': {'value': 5.2239494,
     'description': 'Score based on 1 child docs in range from 0 to 36, using score mode A

In [289]:
text ='Stability c'

resp = es.es.indices.analyze(
    index=idx,
    analyzer="standard",
    text=text,
    filter_path="**.token"
)
resp.body

{'tokens': [{'token': 'stability'}, {'token': 'c'}]}

In [270]:
query

{'nested': {'path': 'completion_terms',
  'query': {'bool': {'should': [{'match_phrase_prefix': {'completion_terms.tag': 'Stability C'}}]}}}}

In [ ]:
should_list = [
                            {
                                "term": {
                                    "completion_terms.tag.edge_ngram": get_completion_prefix(
                                        query_string
                                    )
                                }
                            },
                            {
                                "match_phrase": {
                                    "completion_terms.tag.edge_ngram": {
                                        "query": get_completion_prefix(query_string)
                                    }
                                }
                            },
                            {
                                "match": {
                                    "completion_terms.tag.edge_ngram": {
                                        "query": get_completion_prefix(query_string),
                                        "operator": "OR",
                                        "fuzziness": "AUTO",
                                        "prefix_length": max(
                                            1,
                                            math.floor(
                                                (1 / 2.5)
                                                * len(
                                                    get_completion_prefix(query_string)
                                                )
                                            ),
                                        ),
                                    }
                                }
                            },
                        ]

In [64]:
resp = es.es.search(
    index=idx,
    query=query,
    aggs=aggs,
    size=0,
)
resp.body

{'took': 7,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 4570, 'relation': 'eq'},
  'max_score': None,
  'hits': []},
 'aggregations': {'topics_parent': {'doc_count': 86587,
   'topics': {'doc_count': 8297,
    'topics': {'doc_count_error_upper_bound': 0,
     'sum_other_doc_count': 1744,
     'buckets': [{'key': 'stability control',
       'doc_count': 3723,
       'top_hits': {'hits': {'total': {'value': 3723, 'relation': 'eq'},
         'max_score': 0.0,
         'hits': [{'_index': 'vehicle',
           '_id': 'Tce-rZMBFDxTL3h7vomo',
           '_nested': {'field': 'completion_terms', 'offset': 18},
           '_score': 0.0,
           '_source': {'tag': 'Stability control'},
           'highlight': {'completion_terms.tag.edge_ngram': ['<em>Stability</em> <em>co</em>ntrol']}}]}}},
      {'key': 'air conditioning',
       'doc_count': 812,
       'top_hits': {'hits': {'total': {'value': 812, 'relation': 'eq'}

In [ ]:
stability

In [161]:
for i in range(1,10):
    var = max(1,math.floor((1/2.5) * i))
    print(f"{i}: {var}")


1: 1
2: 1
3: 1
4: 1
5: 2
6: 2
7: 2
8: 3
9: 3


In [ ]:
 math.floor((1/3) * len(get_completion_prefix(query_string)))